Update binary batch train 16 worker 10
- Since the network packet is presented in a 1D format, we
use 1D convolution1
in LuNet, as illustrated in Formula (1)
that specifies the operation to the input vector g with a filter
f of size m.

# Prepare Requirement

In [ ]:
!pip install --upgrade torch torchvision torchaudio
!pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 124.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 105.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
cred = {"username":"nisrinasalm","key":"b740180f8fa2f5861bac033488a60e66"}
! mkdir -p ~/.kaggle
! touch ~/.kaggle/kaggle.json

api_token = cred
import json

with open("/root/.kaggle/kaggle.json", "w") as file:
  json.dump(api_token, file)

! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# !kaggle datasets download -d dhoogla/unswnb15
!kaggle datasets download -d mrwellsdavid/unsw-nb15

Dataset URL: https://www.kaggle.com/datasets/mrwellsdavid/unsw-nb15
License(s): unknown
 82% 122M/149M [00:00<00:00, 348MB/s] 
100% 149M/149M [00:00<00:00, 355MB/s]


In [ ]:
# !unzip unswnb15.zip
!unzip unsw-nb15.zip

Archive:  unsw-nb15.zip
  inflating: NUSW-NB15_features.csv  
  inflating: UNSW-NB15_1.csv         
  inflating: UNSW-NB15_2.csv         
  inflating: UNSW-NB15_3.csv         
  inflating: UNSW-NB15_4.csv         
  inflating: UNSW-NB15_LIST_EVENTS.csv  
  inflating: UNSW_NB15_testing-set.csv  
  inflating: UNSW_NB15_training-set.csv  


In [ ]:
import pandas as pd
import numpy as np
from argparse import Namespace

import torch
torch.set_float32_matmul_precision('medium')
!pip install pytorch-lightning
from pytorch_lightning import LightningModule
from torch import nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.1/823.1 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.6/962.6 kB 61.8 MB/s eta 0:00:00


In [ ]:
# df_train = pd.read_parquet('UNSW_NB15_training-set.parquet')
# df_test = pd.read_parquet('UNSW_NB15_testing-set.parquet')
df_train = pd.read_csv('UNSW_NB15_training-set.csv')
df_test = pd.read_csv('UNSW_NB15_testing-set.csv')

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82332 entries, 0 to 82331
Data columns (total 45 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 82332 non-null  int64  
 1   dur                82332 non-null  float64
 2   proto              82332 non-null  object 
 3   service            82332 non-null  object 
 4   state              82332 non-null  object 
 5   spkts              82332 non-null  int64  
 6   dpkts              82332 non-null  int64  
 7   sbytes             82332 non-null  int64  
 8   dbytes             82332 non-null  int64  
 9   rate               82332 non-null  float64
 10  sttl               82332 non-null  int64  
 11  dttl               82332 non-null  int64  
 12  sload              82332 non-null  float64
 13  dload              82332 non-null  float64
 14  sloss              82332 non-null  int64  
 15  dloss              82332 non-null  int64  
 16  sinpkt             823

In [ ]:
df_train.shape

(82332, 45)

In [ ]:
df_train.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.000011,udp,-,INT,2,0,496,0,90909.0902,...,1,2,0,0,0,1,2,0,Normal,0
1,2,0.000008,udp,-,INT,2,0,1762,0,125000.0003,...,1,2,0,0,0,1,2,0,Normal,0
2,3,0.000005,udp,-,INT,2,0,1068,0,200000.0051,...,1,3,0,0,0,1,3,0,Normal,0
3,4,0.000006,udp,-,INT,2,0,900,0,166666.6608,...,1,3,0,0,0,2,3,0,Normal,0
4,5,0.000010,udp,-,INT,2,0,2126,0,100000.0025,...,1,3,0,0,0,2,3,0,Normal,0


In [ ]:
ATTACK_CAT_TO_ID = {
    'Normal': 0,
    'Reconnaissance': 1,
    'Backdoor': 2,
    'DoS': 3,
    'Exploits': 4,
    'Analysis': 5,
    'Fuzzers': 6,
    'Worms': 7,
    'Shellcode': 8,
    'Generic': 9,
}

In [ ]:
def proto_to_one_hot(x):
    proto_val = [
        'udp', 'arp', 'tcp', 'igmp', 'ospf', 'sctp', 'gre', 'ggp', 'ip', 'ipnip', 'st2', 'argus', 'chaos', 'egp',
        'emcon', 'nvp', 'pup', 'xnet', 'mux', 'dcn', 'hmp', 'prm', 'trunk-1', 'trunk-2', 'xns-idp', 'leaf-1', 'leaf-2',
        'irtp', 'rdp', 'netblt', 'mfe-nsp', 'merit-inp', '3pc', 'idpr', 'ddp', 'idpr-cmtp', 'tp++', 'ipv6', 'sdrp',
        'ipv6-frag', 'ipv6-route', 'idrp', 'mhrp', 'i-nlsp', 'rvd', 'mobile', 'narp', 'skip', 'tlsp', 'ipv6-no', 'any',
        'ipv6-opts', 'cftp', 'sat-expak', 'ippc', 'kryptolan', 'sat-mon', 'cpnx', 'wsn', 'pvp', 'br-sat-mon', 'sun-nd',
        'wb-mon', 'vmtp', 'ttp', 'vines', 'nsfnet-igp', 'dgp', 'eigrp', 'tcf', 'sprite-rpc', 'larp', 'mtp', 'ax.25',
        'ipip', 'aes-sp3-d', 'micp', 'encap', 'pri-enc', 'gmtp', 'ifmp', 'pnni', 'qnx', 'scps', 'cbt', 'bbn-rcc', 'igp',
        'bna', 'swipe', 'visa', 'ipcv', 'cphb', 'iso-tp4', 'wb-expak', 'sep', 'secure-vmtp', 'xtp', 'il', 'rsvp',
        'unas', 'fc', 'iso-ip', 'etherip', 'pim', 'aris', 'a/n', 'ipcomp', 'snp', 'compaq-peer', 'ipx-n-ip', 'pgm',
        'vrrp', 'l2tp', 'zero', 'ddx', 'iatp', 'stp', 'srp', 'uti', 'sm', 'smp', 'isis', 'ptp', 'fire', 'crtp', 'crudp',
        'sccopmce', 'iplt', 'pipe', 'sps', 'ib', 'icmp', 'rtp'
    ]

    one_hot = [0] * len(proto_val)
    one_hot[proto_val.index(x)] = 1

    return one_hot

In [ ]:
def service_to_one_hot(x):
    service_val = [
        '-', 'http', 'ftp', 'ftp-data', 'smtp', 'pop3', 'dns', 'snmp', 'ssl', 'dhcp', 'irc', 'radius', 'ssh'
    ]

    one_hot = [0] * len(service_val)
    one_hot[service_val.index(x)] = 1

    return one_hot

In [ ]:
def state_to_one_hot(x):
    state_val = ['INT', 'FIN', 'REQ', 'ACC', 'CON', 'RST', 'CLO', 'ECO', 'PAR', 'URN', 'no']

    one_hot = [0] * len(state_val)
    one_hot[state_val.index(x)] = 1

    return one_hot

In [ ]:
def double_sided_log(x):
    return np.sign(x) * np.log(1 + np.abs(x))

In [ ]:
def sigmoid(x):
    return np.divide(1, (1 + np.exp(np.negative(x))))

In [ ]:
def preprocess_data(data):
    # extract label and drop id
    attack_cat = data['attack_cat'].apply(func=(lambda x: ATTACK_CAT_TO_ID.get(x))).to_numpy().reshape(-1, 1)
    label = data['label'].to_numpy().reshape(-1, 1)
    data = data.drop(['id', 'attack_cat', 'label'], axis=1)

    # categorical to one-hot-encoding
    # extract and transform categorical to one-hot
    proto = np.array(data['proto'].apply(lambda x: proto_to_one_hot(x)).to_list())
    service = np.array(data['service'].apply(lambda x: service_to_one_hot(x)).to_list())
    state = np.array(data['state'].apply(lambda x: state_to_one_hot(x)).to_list())
    categorical = np.concatenate([proto, service, state], axis=1)
    data = data.drop(['proto', 'service', 'state'], axis=1)

    # transform to np matrix
    data = data.to_numpy()

    # merge categorical feature back to data
    data = np.concatenate([data, categorical], axis=1)

    # double-sided log: to attenuate outliers
    data = double_sided_log(data)

    # sigmoid transform
    data = sigmoid(data)

    # merge label back to data
    data = np.concatenate([data, attack_cat, label], axis=1)

    return data

In [ ]:
train_data = preprocess_data(df_train)
test_data = preprocess_data(df_test)

In [ ]:
import numpy as np
from torch.utils.data import Dataset


class UNSWNB15Dataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        attack_cat = self.data[idx, -2]
        label = self.data[idx, -1]
        feature = self.data[idx, 0:-2]

        # expand dims for matching batch & feature vector
        feature = np.expand_dims(feature, axis=0)

        return {
            'feature': feature,
            'attack_cat': attack_cat,
            'label': label
        }

In [ ]:
class LuNetBlock(LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(
                in_channels=1,
                out_channels=hparams.conv_out,
                kernel_size=hparams.kernel
            ),
            nn.ReLU()
        )
        self.max_pool = nn.Sequential(
            nn.MaxPool1d(kernel_size=2),
            nn.ReLU()
        )
        # calc the output size after max_pool
        dummy_x = torch.randn(1, 1, hparams.input_dim, requires_grad=False)
        dummy_x = self.conv(dummy_x)
        dummy_x = self.max_pool(dummy_x)
        max_pool_out = dummy_x.shape[1]
        lstm_in = dummy_x.shape[2]

        self.batch_norm = nn.BatchNorm1d(max_pool_out)

        self.lstm = nn.LSTM(lstm_in, hparams.conv_out)

        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = self.conv(x)
        x = self.max_pool(x)
        x = self.batch_norm(x)

        # lstm and relu
        x, hidden = self.lstm(x)
        x = F.relu(x)

        # reshape
        x = x.view(x.shape[0], 1, -1)

        # drop out
        x = self.dropout(x)

        return x

In [ ]:
class LuNet(LightningModule):
    def __init__(self, input_dim, c1_out, c1_kernel,
                 c2_out, c2_kernel,
                 c3_out, c3_kernel,
                 final_conv, final_kernel,
                 out, train_data=None, test_data=None):
        super().__init__()

        # Simpan semua parameter kecuali data
        self.save_hyperparameters(ignore=["train_data", "test_data"])

        # Simpan data dan output dimension
        self.train_data = train_data
        self.test_data = test_data
        self.out_dim = out

        # LuNet Block 1
        hparams_lu_block_1 = Namespace(
            input_dim=input_dim,
            conv_out=c1_out,
            kernel=c1_kernel
        )
        self.lu_block_1 = LuNetBlock(hparams_lu_block_1)

        dummy_x = torch.randn(1, 1, input_dim, requires_grad=False)
        dummy_x = self.lu_block_1(dummy_x)

        # LuNet Block 2
        hparams_lu_block_2 = Namespace(
            input_dim=dummy_x.shape[2],
            conv_out=c2_out,
            kernel=c2_kernel
        )
        self.lu_block_2 = LuNetBlock(hparams_lu_block_2)
        dummy_x = self.lu_block_2(dummy_x)

        # LuNet Block 3
        hparams_lu_block_3 = Namespace(
            input_dim=dummy_x.shape[2],
            conv_out=c3_out,
            kernel=c3_kernel
        )
        self.lu_block_3 = LuNetBlock(hparams_lu_block_3)
        dummy_x = self.lu_block_3(dummy_x)

        # Final conv + pooling
        self.conv = nn.Sequential(
              nn.Conv1d(
                  in_channels=1,
                  out_channels=final_conv,
                  kernel_size=final_kernel,
              )
          )

        dummy_x = self.conv(dummy_x)

        self.avg_pool = nn.AvgPool1d(kernel_size=final_conv)
        dummy_x = self.avg_pool(dummy_x)

        self.drop_out = nn.Dropout(p=0.5)

        # Output layer
        if self.out_dim == 2:
            self.out = nn.Sequential(
                nn.Linear(dummy_x.shape[1] * dummy_x.shape[2], 1),
                nn.Sigmoid()
            )
        else:
            self.out = nn.Linear(dummy_x.shape[1] * dummy_x.shape[2], self.out_dim)

    def forward(self, x):
        x = self.lu_block_1(x)
        x = self.lu_block_2(x)
        x = self.lu_block_3(x)
        x = self.conv(x)
        x = self.avg_pool(x)
        x = self.drop_out(x)
        x = x.view(x.size(0), -1)
        return self.out(x)

    def train_dataloader(self):
        return DataLoader(UNSWNB15Dataset(self.train_data), batch_size=16, shuffle=True, num_workers=10)

    def val_dataloader(self):
        return DataLoader(UNSWNB15Dataset(self.test_data), batch_size=16, shuffle=True, num_workers=10)

    def training_step(self, batch, batch_idx):
        x = batch['feature'].float()
        y_hat = self(x)

        if self.out_dim == 2:
            y = batch['label'].float().view(-1, 1)
            loss = F.binary_cross_entropy(y_hat, y)
        else:
            y = batch['attack_cat'].long()
            loss = F.cross_entropy(y_hat, y)

        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x = batch['feature'].float()
        y_hat = self(x)

        if self.out_dim == 2:
            y = batch['label'].float().view(-1, 1)
            loss = F.binary_cross_entropy(y_hat, y)
        else:
            y = batch['attack_cat'].long()
            loss = F.cross_entropy(y_hat, y)

        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.RMSprop(self.parameters(), lr=0.001)


In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pathlib import Path
import glob
import os
# torch.set_float32_matmul_precision('medium')

def train_lunet(input_dim, c1_out, c1_kernel, c2_out, c2_kernel, c3_out, c3_kernel,
                final_conv, final_kernel, out_dim,
                train_data, test_data, num_epoch, gpu,
                model_path, logger=None):
    # ------- hparams & model --------------------------------------------------
    hparams = Namespace(**{
        'input_dim'   : input_dim,
        'c1_out'      : c1_out,
        'c1_kernel'   : c1_kernel,
        'c2_out'      : c2_out,
        'c2_kernel'   : c2_kernel,
        'c3_out'      : c3_out,
        'c3_kernel'   : c3_kernel,
        'final_conv'  : final_conv,
        'final_kernel': final_kernel,
        'out'         : out_dim,
        'train_data'  : train_data,
        'test_data'   : test_data,
    })
    model = LuNet(
        input_dim=hparams.input_dim,
        c1_out=hparams.c1_out,
        c1_kernel=hparams.c1_kernel,
        c2_out=hparams.c2_out,
        c2_kernel=hparams.c2_kernel,
        c3_out=hparams.c3_out,
        c3_kernel=hparams.c3_kernel,
        final_conv=hparams.final_conv,
        final_kernel=hparams.final_kernel,
        out=hparams.out,
        train_data=hparams.train_data,
        test_data=hparams.test_data
    )

    # ------- checkpoint callback (simpan setiap epoch) ------------------------
    ckpt_dir = Path(model_path).with_suffix('')        # folder “model_path/”
    ckpt_dir.mkdir(parents=True, exist_ok=True)

    checkpoint_cb = ModelCheckpoint(
        dirpath = ckpt_dir,
        filename = "lunet-{epoch:02d}",   # lunet-01.ckpt, lunet-02.ckpt, ...
        every_n_epochs = 1,               # simpan setiap epoch
        save_on_train_epoch_end = True,
        save_top_k = -1                   # -1 = simpan SEMUA checkpoint
    )

    # ------- cari checkpoint terakhir (jika ada) ------------------------------
    checkpoints = sorted(glob.glob(str(ckpt_dir / "lunet-*.ckpt")), key=os.path.getmtime)
    resume_checkpoint = checkpoints[-1] if checkpoints else None
    if resume_checkpoint:
        print(f"Resuming from checkpoint: {resume_checkpoint}")
    else:
        print("Starting training from scratch...")

    # ------- trainer ----------------------------------------------------------
    trainer = Trainer(
        max_epochs = num_epoch,
        devices    = gpu,
        #logger     = logger,
        callbacks  = [checkpoint_cb],
        enable_progress_bar=True,
        # resume_from_checkpoint = resume_checkpoint
    )
    trainer.fit(model, ckpt_path=resume_checkpoint)

    # ------- (opsional) simpan checkpoint terakhir dgn nama “latest.ckpt” -----
    latest_ckpt = Path(model_path).absolute()
    trainer.save_checkpoint(str(latest_ckpt))


In [ ]:
def train_lunet_binary(train_data, test_data, gpu, model_path):
    # prepare logger
    logger = TensorBoardLogger('lunet_binary_model_logs', 'lunet_binary_model')

    train_lunet(
        input_dim=196, c1_out=64, c1_kernel=64, c2_out=128, c2_kernel=128, c3_out=256, c3_kernel=256, final_conv=512,
        final_kernel=512, out_dim=2, train_data=train_data, test_data=test_data, num_epoch=10,
        gpu=gpu, model_path=model_path, logger=logger
    )

In [ ]:
def train_lunet_multiclass(train_data, test_data, gpu, model_path):
    # prepare logger
    logger = TensorBoardLogger('lunet_multiclass_model_logs', 'lunet_multiclass_model')

    train_lunet(
        input_dim=196, c1_out=64, c1_kernel=64, c2_out=128, c2_kernel=128, c3_out=256, c3_kernel=256, final_conv=512,
        final_kernel=512, out_dim=10, train_data=train_data, test_data=test_data, num_epoch=10,
        gpu=gpu, model_path=model_path, logger=logger
    )

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import torch
print("Using device:", torch.cuda.get_device_name(0))

Using device: Tesla T4


In [ ]:
gpu = 1
train_lunet_multiclass(train_data=train_data, test_data=test_data, gpu=gpu, model_path='/content/drive/MyDrive/lunet')
# train_lunet_multiclass(train_data_path=train_data_path, test_data_path=val_data_path, gpu=gpu, model_path=model_path)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type       | Params | Mode 
--------------------------------------------------
0 | lu_block_1 | LuNetBlock | 38.1 K | train
1 | lu_block_2 | LuNetBlock | 1.1 M  | train
2 | lu_block_3 | LuNetBlock | 8.6 M  | train
3 | conv       | Sequential | 262 K  | train
4 | avg_pool   | AvgPool1d  | 0      | train
5 | drop_out   | Dropout    | 0      | train
6 | out        | Linear     | 650 K  | train
--------------------------------------------------
10.6 M    Trainable params
0         Non-trainable params
10.6 M    Total params
42.553    Total estimated model params size (MB)
35        Modules in train mode


Starting training from scratch...


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


# Binary

## Batch Train 16

In [ ]:
model = LuNet.load_from_checkpoint(
    "/content/drive/MyDrive/lunet_bin_16/lunet-epoch=09.ckpt",
    input_dim=196,
    c1_out=64, c1_kernel=64,
    c2_out=128, c2_kernel=128,
    c3_out=256, c3_kernel=256,
    final_conv=512,       # ← HARUS 512 agar cocok
    final_kernel=512,     # ← HARUS 512 agar cocok
    out=2,
    train_data=train_data,
    test_data=test_data
)



In [ ]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
import numpy as np
from tqdm import tqdm
import time

# Pastikan model di GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

# Dataloader
# Reduced batch size to mitigate OutOfMemoryError
test_loader = DataLoader(UNSWNB15Dataset(test_data), batch_size=64, shuffle=False)

# Timer mulai
start_time = time.time()

# Kumpulkan semua prediksi dan label
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating"):
        x = batch['feature'].float().to(device)
        y = batch['label'].long().to(device)

        outputs = model(x)
        preds = (outputs > 0.5).int().view(-1)

        all_preds.append(preds)
        all_labels.append(y)

# Gabungkan semua batch
all_preds = torch.cat(all_preds).cpu().numpy()
all_labels = torch.cat(all_labels).cpu().numpy()


Evaluating: 100%|██████████| 2740/2740 [1:13:30<00:00,  1.61s/it]


In [ ]:
# Metrik evaluasi dari scikit-learn
accuracy = accuracy_score(all_labels, all_preds)
detection_rate = recall_score(all_labels, all_preds, average='binary')

cm = confusion_matrix(all_labels, all_preds)
TN, FP, FN, TP = cm.ravel()
false_positive_rate = FP / (FP + TN + 1e-10)

# Tampilkan hasil
print(f"\n📊 Evaluation Metrics (scikit-learn):")
print(f"Accuracy            : {accuracy:.4f}")
print(f"Detection Rate (DR) : {detection_rate:.4f}")
print(f"False Pos. Rate     : {false_positive_rate:.4f}")
print(f"Execution Time      : {time.time() - start_time:.2f} seconds")


📊 Evaluation Metrics (scikit-learn):
Accuracy            : 0.6806
Detection Rate (DR) : 1.0000
False Pos. Rate     : 1.0000
Execution Time      : 4467.60 seconds


## Batch Train 8

In [ ]:
gpu = 1
train_lunet_binary(train_data=train_data, test_data=test_data, gpu=gpu, model_path='/content/drive/MyDrive/lunet_bin_16')
# train_lunet_multiclass(train_data_path=train_data_path, test_data_path=val_data_path, gpu=gpu, model_path=model_path)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Starting training from scratch...


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type       | Params | Mode 
--------------------------------------------------
0 | lu_block_1 | LuNetBlock | 38.1 K | train
1 | lu_block_2 | LuNetBlock | 1.1 M  | train
2 | lu_block_3 | LuNetBlock | 8.6 M  | train
3 | conv       | Sequential | 262 K  | train
4 | avg_pool   | AvgPool1d  | 0      | train
5 | drop_out   | Dropout    | 0      | train
6 | out        | Sequential | 65.0 K | train
--------------------------------------------------
10.1 M    Trainable params
0         Non-trainable params
10.1 M    Total params
40.212    Total estimated model params size (MB)
37        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:476: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
model = LuNet.load_from_checkpoint(
    "/content/drive/MyDrive/lunet-epoch=09.ckpt",
    input_dim=196,
    c1_out=64, c1_kernel=64,
    c2_out=128, c2_kernel=128,
    c3_out=256, c3_kernel=256,
    final_conv=512,       # ← HARUS 512 agar cocok
    final_kernel=512,     # ← HARUS 512 agar cocok
    out=2,
    train_data=train_data,
    test_data=test_data
)



In [ ]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
import numpy as np
from tqdm import tqdm
import time

# Pastikan model di GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

# Dataloader
# Reduced batch size to mitigate OutOfMemoryError
test_loader = DataLoader(UNSWNB15Dataset(test_data), batch_size=64, shuffle=False)

# Timer mulai
start_time = time.time()

# Kumpulkan semua prediksi dan label
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating"):
        x = batch['feature'].float().to(device)
        y = batch['label'].long().to(device)

        outputs = model(x)
        preds = (outputs > 0.5).int().view(-1)

        all_preds.append(preds)
        all_labels.append(y)

# Gabungkan semua batch
all_preds = torch.cat(all_preds).cpu().numpy()
all_labels = torch.cat(all_labels).cpu().numpy()


In [ ]:
# Metrik evaluasi dari scikit-learn
accuracy = accuracy_score(all_labels, all_preds)
detection_rate = recall_score(all_labels, all_preds, average='binary')

cm = confusion_matrix(all_labels, all_preds)
TN, FP, FN, TP = cm.ravel()
false_positive_rate = FP / (FP + TN + 1e-10)

# Tampilkan hasil
print(f"\n📊 Evaluation Metrics (scikit-learn):")
print(f"Accuracy            : {accuracy:.4f}")
print(f"Detection Rate (DR) : {detection_rate:.4f}")
print(f"False Pos. Rate     : {false_positive_rate:.4f}")
print(f"Execution Time      : {time.time() - start_time:.2f} seconds")


📊 Evaluation Metrics (scikit-learn):
Accuracy            : 0.8318
Detection Rate (DR) : 0.7578
False Pos. Rate     : 0.0105
Execution Time      : 4318.45 seconds


# Multiclass

In [ ]:
model = LuNet.load_from_checkpoint(
    "/content/drive/MyDrive/lunet/lunet-epoch=09.ckpt",
    input_dim=196,
    c1_out=64, c1_kernel=64,
    c2_out=128, c2_kernel=128,
    c3_out=256, c3_kernel=256,
    final_conv=512,       # ← HARUS 512 agar cocok
    final_kernel=512,     # ← HARUS 512 agar cocok
    out=10,
    train_data=train_data,
    test_data=test_data
)


## Batch Train 16, Batch Test 128

In [ ]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
import numpy as np
from tqdm import tqdm
import time

# Pastikan model di GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

# Dataloader
# Reduced batch size to mitigate OutOfMemoryError
test_loader = DataLoader(UNSWNB15Dataset(test_data), batch_size=128, shuffle=False)

# Timer mulai
start_time = time.time()

# Kumpulkan semua prediksi dan label
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating"):
        x = batch['feature'].float().to(device)
        y = batch['attack_cat'].long().to(device)

        outputs = model(x)
        preds = torch.argmax(outputs, dim=1)

        all_preds.append(preds)
        all_labels.append(y)

# Gabungkan semua batch
all_preds = torch.cat(all_preds).cpu().numpy()
all_labels = torch.cat(all_labels).cpu().numpy()

# Metrik evaluasi dari scikit-learn
accuracy = accuracy_score(all_labels, all_preds)
detection_rate = recall_score(all_labels, all_preds, average='macro')  # Macro average recall = DR
false_positive_rate_per_class = []
cm = confusion_matrix(all_labels, all_preds)

for i in range(len(cm)):
    FP = np.sum(cm[:, i]) - cm[i, i]
    TN = np.sum(cm) - (np.sum(cm[i, :]) + np.sum(cm[:, i]) - cm[i, i])
    fpr = FP / (FP + TN + 1e-10)
    false_positive_rate_per_class.append(fpr)

false_positive_rate = np.mean(false_positive_rate_per_class)

# Tampilkan hasil
print(f"\n📊 Evaluation Metrics (scikit-learn):")
print(f"Accuracy            : {accuracy:.4f}")
print(f"Detection Rate (DR) : {detection_rate:.4f}")
print(f"False Pos. Rate     : {false_positive_rate:.4f}")
print(f"Execution Time      : {time.time() - start_time:.2f} seconds")

Evaluating: 100%|██████████| 1370/1370 [06:22<00:00,  3.58it/s]



📊 Evaluation Metrics (scikit-learn):
Accuracy            : 0.6220
Detection Rate (DR) : 0.2946
False Pos. Rate     : 0.0456
Execution Time      : 382.92 seconds


In [ ]:
!nvidia-smi


Fri Jun 13 02:53:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             50W /  400W |   33085MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

## Batch Train 16, Batch Test 64

In [ ]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
import numpy as np
from tqdm import tqdm
import time

# Pastikan model di GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

# Dataloader
# Reduced batch size to mitigate OutOfMemoryError
test_loader = DataLoader(UNSWNB15Dataset(test_data), batch_size=64, shuffle=False)

# Timer mulai
start_time = time.time()

# Kumpulkan semua prediksi dan label
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating"):
        x = batch['feature'].float().to(device)
        y = batch['attack_cat'].long().to(device)

        outputs = model(x)
        preds = torch.argmax(outputs, dim=1)

        all_preds.append(preds)
        all_labels.append(y)

# Gabungkan semua batch
all_preds = torch.cat(all_preds).cpu().numpy()
all_labels = torch.cat(all_labels).cpu().numpy()

# Metrik evaluasi dari scikit-learn
accuracy = accuracy_score(all_labels, all_preds)
detection_rate = recall_score(all_labels, all_preds, average='macro')  # Macro average recall = DR
false_positive_rate_per_class = []
cm = confusion_matrix(all_labels, all_preds)

for i in range(len(cm)):
    FP = np.sum(cm[:, i]) - cm[i, i]
    TN = np.sum(cm) - (np.sum(cm[i, :]) + np.sum(cm[:, i]) - cm[i, i])
    fpr = FP / (FP + TN + 1e-10)
    false_positive_rate_per_class.append(fpr)

false_positive_rate = np.mean(false_positive_rate_per_class)

# Tampilkan hasil
print(f"\n📊 Evaluation Metrics (scikit-learn):")
print(f"Accuracy            : {accuracy:.4f}")
print(f"Detection Rate (DR) : {detection_rate:.4f}")
print(f"False Pos. Rate     : {false_positive_rate:.4f}")
print(f"Execution Time      : {time.time() - start_time:.2f} seconds")

Evaluating: 100%|██████████| 2740/2740 [1:08:02<00:00,  1.49s/it]



📊 Evaluation Metrics (scikit-learn):
Accuracy            : 0.6200
Detection Rate (DR) : 0.2947
False Pos. Rate     : 0.0457
Execution Time      : 4083.78 seconds


## Batch Train 8

In [ ]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
import numpy as np
from tqdm import tqdm
import time

# Pastikan model di GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

# Dataloader
test_loader = DataLoader(UNSWNB15Dataset(test_data), batch_size=64, shuffle=False)

# Timer mulai
start_time = time.time()

# Kumpulkan semua prediksi dan label
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating"):
        x = batch['feature'].float().to(device)
        y = batch['attack_cat'].long().to(device)

        outputs = model(x)
        preds = torch.argmax(outputs, dim=1)

        all_preds.append(preds)
        all_labels.append(y)

# Gabungkan semua batch
all_preds = torch.cat(all_preds).cpu().numpy()
all_labels = torch.cat(all_labels).cpu().numpy()

# Metrik evaluasi dari scikit-learn
accuracy = accuracy_score(all_labels, all_preds)
detection_rate = recall_score(all_labels, all_preds, average='macro')  # Macro average recall = DR
false_positive_rate_per_class = []
cm = confusion_matrix(all_labels, all_preds)

for i in range(len(cm)):
    FP = np.sum(cm[:, i]) - cm[i, i]
    TN = np.sum(cm) - (np.sum(cm[i, :]) + np.sum(cm[:, i]) - cm[i, i])
    fpr = FP / (FP + TN + 1e-10)
    false_positive_rate_per_class.append(fpr)

false_positive_rate = np.mean(false_positive_rate_per_class)

# Tampilkan hasil
print(f"\n📊 Evaluation Metrics (scikit-learn):")
print(f"Accuracy            : {accuracy:.4f}")
print(f"Detection Rate (DR) : {detection_rate:.4f}")
print(f"False Pos. Rate     : {false_positive_rate:.4f}")
print(f"Execution Time      : {time.time() - start_time:.2f} seconds")


Evaluating: 100%|██████████| 685/685 [30:56<00:00,  2.71s/it]



📊 Evaluation Metrics (scikit-learn):
Accuracy            : 0.4775
Detection Rate (DR) : 0.1901
False Pos. Rate     : 0.0660
Execution Time      : 1858.77 seconds


In [ ]:
model.eval()
model = model.to("cuda" if torch.cuda.is_available() else "cpu")

test_loader = DataLoader(UNSWNB15Dataset(test_data), batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for batch in test_loader:
        x = batch['feature'].float().to(model.device)
        y = batch['attack_cat'].long().to(model.device)  # multiclass label

        outputs = model(x)  # shape: (batch_size, num_classes)
        preds = torch.argmax(outputs, dim=1)  # ambil kelas dengan skor tertinggi

        correct += (preds == y).sum().item()
        total += y.size(0)

accuracy = correct / total
print(f"Akurasi multiclass: {accuracy:.4f}")

In [ ]:
test_loader = DataLoader(UNSWNB15Dataset(test_data), batch_size=64, shuffle=False)


In [ ]:
model.eval()
model = model.to("cuda" if torch.cuda.is_available() else "cpu")

test_loader = DataLoader(UNSWNB15Dataset(test_data), batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for batch in test_loader:
        x = batch['feature'].float().to(model.device)
        y = batch['label'].long().to(model.device)

        outputs = model(x)
        preds = (outputs > 0.5).int().view(-1)  # binary classification
        correct += (preds == y).sum().item()
        total += y.size(0)

accuracy = correct / total
print(f"Akurasi: {accuracy:.4f}")
